In [2]:
!pip install transformers datasets

     |████████████████████████████████| 290 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 4.4 MB/s eta 0:00:01
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
     |████████████████████████████████| 19.1 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 574 kB 5.0 MB/s eta 0:00:01
  Using cached charset_normalizer-2.0.7-py3-none-any.whl (38 kB)
     |████████████████████████████████| 81 kB 7.0 MB/s eta 0:00:011
     |████████████████████████████████| 45 kB 4.0 MB/s eta 0:00:011
     |████████████████████████████████| 121 kB 5.8 MB/s eta 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
  Attempting uninstall: fsspec
    Found existing ins

In [3]:
import datasets

In [6]:
from datasets import load_dataset

dataset = load_dataset("reuters21578", "ModHayes")

Reusing dataset reuters21578 (/Users/markmcquade/.cache/huggingface/datasets/reuters21578/ModHayes/1.0.0/bd91fac5a25fc818873c02a7281cc276c9b326a9e6a89288fc6ba6967772240f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'text_type', 'topics', 'lewis_split', 'cgis_split', 'old_id', 'new_id', 'places', 'people', 'orgs', 'exchanges', 'date', 'title'],
        num_rows: 722
    })
    train: Dataset({
        features: ['text', 'text_type', 'topics', 'lewis_split', 'cgis_split', 'old_id', 'new_id', 'places', 'people', 'orgs', 'exchanges', 'date', 'title'],
        num_rows: 20856
    })
})

Remove unwanted columns

In [8]:
dataset = dataset.remove_columns(
['text_type', 'topics', 'lewis_split', 'cgis_split', 'old_id', 
 'new_id', 'places', 'people', 'orgs', 'exchanges', 'date'
])

dataset = dataset.rename_column('title', 'target')

In [9]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'target'],
        num_rows: 722
    })
    train: Dataset({
        features: ['text', 'target'],
        num_rows: 20856
    })
})

In [20]:
dataset['train'][1]

{'text': 'Standard Oil Co and BP North America\nInc said they plan to form a venture to manage the money market\nborrowing and investment activities of both companies.\n    BP North America is a subsidiary of British Petroleum Co\nPlc &lt;BP>, which also owns a 55 pct interest in Standard Oil.\n    The venture will be called BP/Standard Financial Trading\nand will be operated by Standard Oil under the oversight of a\njoint management committee.\n\n Reuter\n',
 'target': 'STANDARD OIL &lt;SRD> TO FORM FINANCIAL UNIT'}

Drop new lines and tabs, replace with white space. Remove commas, remove quotes. Replace "Reuter" from end of text. Drop extra white space agains. Remove html tags

In [22]:
def clean(row):
    row['text'] = row['text'].replace('\n',' ').replace('\t',' ')\
                             .replace(',','').replace('\'','').replace('\"','')\
                             .replace(' Reuter','').replace(' REUTER','')
    row['text'] = " ".join(row['text'].split())
    row['target'] = row['target'].replace('&lt;','<').replace('&gt;','>')
    return row

In [23]:
dataset = dataset.map(clean)

  0%|          | 0/722 [00:00<?, ?ex/s]

  0%|          | 0/20856 [00:00<?, ?ex/s]

In [24]:
dataset['train'][1]

{'text': 'Standard Oil Co and BP North America Inc said they plan to form a venture to manage the money market borrowing and investment activities of both companies. BP North America is a subsidiary of British Petroleum Co Plc &lt;BP> which also owns a 55 pct interest in Standard Oil. The venture will be called BP/Standard Financial Trading and will be operated by Standard Oil under the oversight of a joint management committee.',
 'target': 'STANDARD OIL <SRD> TO FORM FINANCIAL UNIT'}

In [25]:
dataset.save_to_disk('reuters_processed')

In [27]:
dataset['train'].to_csv('reuters_train.csv', index=False, header=True)
dataset['test'].to_csv('reuters_test.csv', index=False, header=True)

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

807451